In [3]:
import boto3,time
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
TABLE_NAME = "dataset_info"
dynamodb = boto3.resource('dynamodb',
                          region_name="us-west-2"
)

table = dynamodb.Table(TABLE_NAME)

response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
cust_schema = """ dataset_key  string, created_ts  string, data_layer  string, data_location  string, dataset_id  string, dataset_refresh_end_ts  string, dataset_refresh_start_ts  string, full_load_start_ts  string, last_update_ts  string, run_mode  string, sla string, source_name  string, watermark_ts string """

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = spark.createDataFrame(data,cust_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df = df.withColumn("data_location", when((df.data_location.isNull() | (df.data_location == ' ') | (df.data_location == '')) ,'null').otherwise(col("data_location")))
df = df.withColumn("full_load_start_ts", when((df.full_load_start_ts.isNull() | (df.full_load_start_ts == " ") | (df.full_load_start_ts == '')) ,"null").otherwise(col("full_load_start_ts")))
df = df.withColumn("run_mode", when((df.run_mode.isNull() | (df.run_mode == ' ') | (df.run_mode == '')) ,'null').otherwise(col("run_mode")))
df = df.withColumn("sla", when((df.sla.isNull() | (df.sla == ' ') | (df.sla == '')) ,'-1').otherwise(col("sla")))
df = df.withColumn("watermark_ts", when((df.watermark_ts.isNull() | (df.watermark_ts == ' ') | (df.watermark_ts == '')) ,'null').otherwise(col("watermark_ts")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
start = time.time()
df.coalesce(1).write.mode("overwrite").option("header","true").csv("s3://gd-emeadataunif-dev-private-gde-audit-report/dataset_info/")
end = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
end-start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5.468175649642944

####  job_run_details

In [10]:
TABLE_NAME = "job_run_details"

dynamodb = boto3.resource('dynamodb', region_name="us-west-2")
table = dynamodb.Table(TABLE_NAME)
response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
cust_schema = """ error_message string, target_records string, dataset_id string, source string, job_key string, source_records string, dataset string, layer_name string, job_name string, end_ts string, start_ts string, job_run_date string, job_status string"""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df = spark.createDataFrame(data,cust_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df = df.withColumn("dataset_id",  when((df.dataset_id.isNull() | (df.dataset_id == "")| (df.dataset_id == " ")) ,'-1').otherwise(col("dataset_id")))
df = df.withColumn("target_records",  when((df.target_records.isNull() | (df.target_records == "")| (df.target_records == " ")) ,'-1').otherwise(col("target_records")))
df = df.withColumn("source_records",  when((df.source_records.isNull() | (df.source_records == "")| (df.source_records == " ")) ,'-1').otherwise(col("source_records")))
df = df.withColumn("start_ts", when((df.start_ts.isNull()| (df.start_ts == '')| (df.start_ts == ' ')),'-1').otherwise(col("start_ts")))
df = df.withColumn("end_ts", when((df.end_ts.isNull() | (df.end_ts == '')| (df.end_ts == ' ')) ,'-1').otherwise(col("end_ts")))
df = df.withColumn("source", when((df.source.isNull() | (df.source == '') | (df.source == ' ')),'null').otherwise(col("source")))
df = df.withColumn("dataset", when((df.dataset.isNull() | (df.dataset == '') | (df.dataset == ' ')),'null').otherwise(col("dataset")))
df = df.withColumn("layer_name", when((df.layer_name.isNull() | (df.layer_name == '') | (df.layer_name == ' ')),'null').otherwise(col("layer_name")))
df = df.withColumn("job_name", when((df.job_name.isNull() | (df.job_name == '') | (df.job_name == ' ')),'null').otherwise(col("job_name")))
df = df.withColumn("job_run_date", when((df.job_run_date.isNull() | (df.job_run_date == '') | (df.job_run_date == ' ')),'null').otherwise(col("job_run_date")))

df = df.withColumn("error_message", when((df.target_records.isNull() | (df.error_message == '') | (df.error_message == ' ')),'null').otherwise(col("error_message")))
df = df.withColumn("error_message", when(df.error_message != 'null',lit('Spark Error')).otherwise(col("error_message")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
start = time.time()
df.coalesce(1).write.mode("overwrite").option("header","true").csv("s3://gd-emeadataunif-dev-private-gde-audit-report/job_run_details/")
end = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
end-start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13.884445190429688